In [1]:
import requests

import time
from requests.exceptions import RequestException

def fetch_data_notebook(url, retries=3, delay=2):
    """
    Fetches data from the given URL with retry and error handling for Jupyter Notebook.

    :param url: The URL to fetch data from.
    :param retries: Number of retries in case of failure.
    :param delay: Delay (in seconds) between retries.
    :return: Response object if successful, or None if all retries fail.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://google.com'
    }

    for attempt in range(retries):
        try:
            print(f"Fetching data from: {url} (Attempt {attempt + 1}/{retries})")
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                print("Data fetched successfully!")
                return response
            else:
                print(f"Unexpected status code {response.status_code}. Retrying...")
        except RequestException as e:
            print(f"Error: {e}. Retrying...")

        time.sleep(delay)
    
    print("Failed to fetch data after multiple retries.")
    return None

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = fetch_data_notebook(standings_url)#requests.get(standings_url)

Fetching data from: https://fbref.com/en/comps/9/Premier-League-Stats (Attempt 1/3)
Data fetched successfully!


In [4]:
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO


In [16]:
years = list(range(2024, 2021, -1))
print(years)
all_matches = []

[2024, 2023, 2022]


In [17]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
counter = 1

In [18]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    print(previous_season)
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        print(f'now is {counter}')
        counter+=1
        time.sleep(1)
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        try:
            shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        except Exception:
            continue
            
        print(shooting)
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(2)

/en/comps/9/2023-2024/2023-2024-Premier-League-Stats
now is 1
   For Liverpool                                                           \
            Date   Time            Comp           Round  Day Venue Result   
0     2024-08-17  12:30  Premier League     Matchweek 1  Sat  Away      W   
1     2024-08-25  16:30  Premier League     Matchweek 2  Sun  Home      W   
2     2024-09-01  16:00  Premier League     Matchweek 3  Sun  Away      W   
3     2024-09-14  15:00  Premier League     Matchweek 4  Sat  Home      L   
4     2024-09-17  21:00    Champions Lg    League phase  Tue  Away      W   
5     2024-09-21  15:00  Premier League     Matchweek 5  Sat  Home      W   
6     2024-09-25  20:00         EFL Cup     Third round  Wed  Home      W   
7     2024-09-28  17:30  Premier League     Matchweek 6  Sat  Away      W   
8     2024-10-02  20:00    Champions Lg    League phase  Wed  Home      W   
9     2024-10-05  12:30  Premier League     Matchweek 7  Sat  Away      W   
10    2024-10-

ValueError: No tables found

In [ ]:
len(all_matches)

In [20]:
match_df = pd.concat(all_matches)

In [21]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [23]:
match_df.to_csv("matches2.csv")